In [5]:
from dotenv import load_dotenv
import os

load_dotenv()



True

In [5]:
import os
import psycopg2
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


# If connection_string is not directly provided above, 
# then check if DATABASE_URL is set in the environment or .env.

connection_string = os.environ.get("NEON_DATABASE_CONNECTION_STRING")

# If neither method provides a connection string, raise an error.
if not connection_string:
    raise ValueError("Please provide a valid connection string either in the code or in the .env file as DATABASE_URL.")

# Connect using the connection string
connection = psycopg2.connect(connection_string)

# Create a new cursor object
cursor = connection.cursor()

In [7]:
# Execute this query to test the database connection
cursor.execute("SELECT 1;")
result = cursor.fetchone()

# Check the query result
if result == (1,):
    print("Your database connection was successful!")
else:
    print("Your connection failed.")
    

Your database connection was successful!


In [33]:
from sqlalchemy.orm import Session
from meme_database.models import MemeEntry

from sqlalchemy import create_engine, select

# Create a new SQLAlchemy engine using the connection string
engine = create_engine(connection_string)

# Get all MemeEntry objects from the database
with Session(engine) as session:
    meme_entries = session.execute(select(MemeEntry)).scalars().all()


In [34]:
len(meme_entries)

7147

In [35]:
# Create a jsonl file
'''
{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
{"custom_id": "request-2", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are an unhelpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
'''
import json

with open("meme_entries.jsonl", "w") as file:
    
    for meme in meme_entries:
        json.dump({
            "custom_id": str(meme.id),
            "method": "POST",
            "url": "/v1/embeddings",
            "body": {
                "model": "text-embedding-3-small",
                "input": meme.content,
                "encoding_format": "float"
            }
        }, file)
        file.write("\n")

In [16]:
# load jsonl file
from sqlalchemy.orm import Session
from meme_database.models import MemeEntry

from sqlalchemy import create_engine, select
import json

# Create a new SQLAlchemy engine using the connection string
engine = create_engine(connection_string)

# Get all MemeEntry objects from the database
with Session(engine) as session:
    with open("openaiembeddings.jsonl", "r") as file:
        for line in file:
            meme_embedding = json.loads(line)
            meme:MemeEntry = session.execute(select(MemeEntry).where(MemeEntry.id == meme_embedding["custom_id"])).scalar()
            meme.content_embedding = meme_embedding['response']["body"]["data"][0]['embedding']
            
            session.add(meme)
        session.commit()
        